In [ ]:
import pandas as pd
import plotly.express as px
import random

subjects = ['Math', 'English', 'Science', 'History']
weeks = [1, 2, 3, 4]

data = {
    'subject': [],
    'stress': [],
    'note': [],
    'week': []
}

for subject in subjects:
    for week in weeks:
        data['subject'].append(subject)
        data['week'].append(week)
        data['stress'].append(random.randint(1, 5))
        data['note'].append(f'Stress note for {subject} in week {week}')


df = pd.DataFrame(data)

# Create the plotly graph
fig = px.line(df, x='week', y='stress', color='subject', hover_data=['note'],
              title='Stress Levels Over Weeks', labels={'week': 'Week', 'stress': 'Stress Level'})

# Update hover template to display the note
fig.update_traces(mode='markers+lines', hovertemplate='Week: %{x}<br>Stress Level: %{y}<br>Note: %{text}',
                  text=df['note'])


# Add legend
fig.update_layout(legend=dict(title='Subject'))

# Show the graph
fig.show()

In [ ]:
def word_wrap(string, n):
    words = string.split()
    lines = []
    current_line = ''
    
    for word in words:
        if len(current_line) + len(word) <= n:
            current_line += word + ' '
        else:
            lines.append(current_line.strip())
            current_line = word + ' '
    
    lines.append(current_line.strip())
    return '<br>'.join(lines)

my_string = "This is a long string that needs to be wrapped into multiple lines with a maximum of 10 characters per line"
wrapped_string = word_wrap(my_string, 10)
print(wrapped_string)

In [ ]:
import pandas as pd
import plotly.express as px

# Read CSV file with the first row as column names
df = pd.read_csv('test_data.csv')

# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

# Define the maximum number of characters before adding a line break
n = 50

# Loop over the note column and add line breaks after the nearest space to the nth character
df['note'] = df['note'].apply(lambda x: word_wrap(x, n))

# Create the plotly graph
fig = px.line(df, x='date', y='stress', color='subject', hover_data=['note', 'label'],
              title='Stress Levels Over Weeks', labels={'date': 'Date', 'stress': 'Stress Level'})
annotations = []
# Add annotations
for subject in df['subject'].unique():
    subject_data = df[df['subject'] == subject]
    for x_val, y_val, label_text in zip(subject_data['date'], subject_data['stress'], subject_data['label']):
        if str(label_text) != "nan":
            arg = dict(x=x_val, y=y_val, text=label_text, showarrow=True, arrowhead=1, yshift=1,
                        font=dict(color='black', size=10), textangle=60, visible=True, name=subject)
            annotations.append(arg)
            
            # fig.add_annotation(x=x_val, y=y_val, text=label_text, showarrow=True, arrowhead=1, yshift=1,
            #                 font=dict(color='black', size=10), textangle=60, visible=True, name=subject)
fig.update_layout(annotations=annotations)
# Add button to toggle annotations visibility
buttons = []
for subject in df['subject'].unique():
    args=[{'annotations': [{**a, 'visible':True if subject == a['name'] else False} for a in annotations]}]
    print(args)
    # Define the button label and method
    button = dict(label=subject, method='relayout', 
                  # Define the button arguments 
                  args=args)
    # Append the button to the list 
    buttons.append(button)

buttons.append(dict(label='All', method='relayout', args=[{'annotations': [{**a, "visible": True} for a in annotations]}]))
buttons.append(dict(label='None', method='relayout', args=[{'annotations': [{**a, "visible": False} for a in annotations]}]))
fig.update_layout(updatemenus=[dict(type='buttons', direction='right', x=0.7, y=1.2, showactive=True, buttons=buttons)])

# Update x-axis tickformat to display only date (without time)
fig.update_xaxes(type='date', tickformat='%Y-%m-%d')

# Update hover template to display the note and formatted date
fig.update_traces(mode='markers+lines', hovertemplate='Date: %{x|%Y-%m-%d}<br>Stress Level: %{y}<br>Note: %{text}',
                  text=df['note'], hoverlabel=dict(namelength=-1))

# Add legend
fig.update_layout(legend=dict(title='Subject'))

# Show the graph
fig.show()